In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, chi2, truncnorm, multivariate_normal
from tqdm import trange
from sklearn import linear_model
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from densratio import densratio
from numpy import linalg as la
import matplotlib.pyplot as plt

# Functions

In [142]:
import numpy as np
from sklearn.linear_model import LogisticRegression

def density_ratio_estimate_prob(D_nu, D_de):
    l_nu = np.ones(len(D_nu))
    l_de = np.zeros(len(D_de))
    
    l = np.concatenate((l_nu, l_de))
    D = np.concatenate((D_nu, D_de))
    
    #fit losgistic model
    C = 0.1
    model = LogisticRegression(penalty= 'l2', C= C)
    model.fit(D, l)
    
    # get density ratios for all samples
    density_ratios = (model.predict_proba(D_de)[:, 1] / model.predict_proba(D_nu)[:, 0])*(len(D_de)/len(D_nu))
    
    return density_ratios

def true_density_ratio(X, Z, V):
    zs_prob = multivariate_normal.pdf(Z, mean=np.zeros(20), cov=np.identity(20))
    vs_prob = multivariate_normal.pdf(V, mean=np.dot(Z,s) - X, cov=np.identity(1))
    zt_prob = multivariate_normal.pdf(Z, mean=0.1*np.ones(20), cov=np.identity(20))
    vt_prob = multivariate_normal.pdf(V, mean=np.dot(Z,s) + X, cov=np.identity(1))
    
    return (zt_prob*vt_prob)/(zs_prob*vs_prob)
    


# def Covariate_Shift_Weight(x, z, v = 0):
#     return np.exp(((x - z @ s)**2 - (x - z @ t)**2)/2)

def Model_X(z, v):
    return z @ u + np.random.normal(0, 1, 1)

def T_statistic(y, x, z, v, regr):
    d_y = regr.predict(z.reshape(1,20))
    d_x = z @ u
    return np.abs((y-d_y)*(x-d_x))
    # return (y - z.sum()- x)**2

def Conterfeits(y, x, z, v, L, K, regr):
    M = L * K - 1
    cnt = 0
    t_stat = T_statistic(y, x, z, v, regr)
    
    for i in range(M):
        x_ = Model_X(z, v)
        if t_stat > T_statistic(y, x_, z, v, regr):
            cnt=cnt+1
            
    return cnt // K

def PCRtest(Y, X, Z, V, L, K, covariate_shift, density_ratio, regr):
    n = Y.size
    W = np.array([0.0]*L)

    for j in range(n):
        y, x, z, v = Y[j], X[j], Z[j], V[j]
        if covariate_shift == True:
            ind = Conterfeits(y, x, z, v, L, K, regr)
            W[ind] += density_ratio[j]
            # W[ind] += true_density_ratio(x, z, v)
        if covariate_shift == False:
            W[Conterfeits(y, x, z, v, L, K, regr)] += 1
    return W, L/n * np.dot(W - n/L, W - n/L)

def generate_cov_matrix(Y, X, Z, V, L, K, density_ratio, regr):
    """
    Generate a covariance matrix for quadratic form normal rv.

    Parameters:
    - L (int): The size of the covariance matrix.

    Returns:
    - covariance_matrix (ndarray): The generated covariance matrix.
    """
    n = Y.size
    diag = np.array([0.0]*L)
    
    for j in range(n):
        y, x, z, v = Y[j], X[j], Z[j], V[j]
        diag[Conterfeits(y, x, z, v, L, K, regr)] += (density_ratio[j]**2)
    diag = L*(diag/n)- 1/L
    covariance_matrix = np.full((L, L), -1/L)  # Fill all entries with 1/L
    np.fill_diagonal(covariance_matrix, diag)  # Set diagonal entries to 1 - 1/L^2
    return covariance_matrix
    

In [143]:
import scipy.stats as stats

def chi_squared_p_value(chi_squared_statistic, df):
    """
    Calculate the p-value from a chi-squared distribution.

    Parameters:
    - chi_squared_statistic (float): The observed chi-squared test statistic.
    - df (int): The degrees of freedom.

    Returns:
    - p_value (float): The calculated p-value.
    """
    p_value = 1 - stats.chi2.cdf(chi_squared_statistic, df)
    return p_value

def monte_carlo_p_value(n_samples, covariance_matrix, L, quantile):
    """
    Calculate the probability corresponding to a given quantile using the Monte Carlo method.

    Parameters:
    - n_samples (int): The number of Monte Carlo samples to generate.
    - covariance_matrix (ndarray): The covariance matrix of the random vector X.
    - L (int): The number of components to sum.
    - quantile (float): The quantile value.

    Returns:
    - probability (float): The estimated probability corresponding to the quantile.
    """
    count = 0
    for _ in range(n_samples):
        sample = np.random.multivariate_normal(np.zeros(L), covariance_matrix)
        squared_sum = np.sum(sample**2)
        if squared_sum <= quantile:
            count += 1

    probability = count / n_samples
    return 1-probability


# 生成数据

In [144]:
# Generate Data

def generate(n, p, s, t, u, Alpha = 0):
    Z_source, Z_target = np.zeros((n, p)), np.zeros((n, p))
    V_source, V_target = 0, 0
    for i in range(n):
        Z_source[i] = np.random.normal(0, 1, p)
        Z_target[i] = np.random.normal(1, 1, p)
        
    X_source = Z_source @ u + np.random.normal(0, 1, n)
    X_target = Z_target @ u + np.random.normal(0.1, 1, n)

    V_source = Z_source @ s + X_source + np.random.normal(0, 1, n)
    V_target = Z_target @ t - X_target + np.random.normal(0, 1, n)
    
    Y_source, Y_target = np.zeros(n), np.zeros(n)
    for i in range(n):
        Y_source[i] = Z_source[i].sum() + X_source[i] + V_source[i] + np.random.normal(0, 1, 1) + Alpha * X_source[i]
        Y_target[i] = Z_target[i].sum() + X_target[i] + V_target[i] + np.random.normal(0, 1, 1) + Alpha * X_target[i]
    return Y_source.reshape(-1,1), X_source.reshape(-1,1), V_source.reshape(-1,1), Z_source,\
           Y_target.reshape(-1,1), X_target.reshape(-1,1), V_target.reshape(-1,1), Z_target

# Test procedure

In [145]:
K = 30
L = 10
n, p = 1000, 20
s = np.random.normal(0, 1, p)
t = s + np.random.normal(0, 0.1, p)
u = np.random.normal(0, 1, p)

In [149]:
#generate data
Y_source, X_source, V_source, Z_source, Y_target, X_target, V_target, Z_target = generate(n, p, s, t, u, 0)

# calculate density ratio
D_s = np.concatenate((X_source, Z_source, V_source), axis = 1)
D_t = np.concatenate((X_target, Z_target, V_target), axis = 1)
densratio_obj = densratio(D_t, D_s)

#calculate density ratio for each sample
sample_density_ratio = densratio_obj.compute_density_ratio(D_s)

# distill information of Z on Y
reg = LassoCV().fit(Z_source,Y_source)


#make random forest for statistic
from sklearn.ensemble import RandomForestRegressor

Ireg = RandomForestRegressor(n_estimators=20)
Ireg.fit(Z_source, Y_source)

#Fit neural network for statistics




RuLSIF starting...
Searching for the optimal sigma and lambda...
sigma = 0.00100, lambda = 0.00100, score = 0.00000
sigma = 0.00100, lambda = 0.01000, score = 0.00000
sigma = 0.00100, lambda = 0.10000, score = 0.00000
sigma = 0.00100, lambda = 1.00000, score = -0.00000
sigma = 0.00100, lambda = 10.00000, score = 0.00000
sigma = 0.00100, lambda = 100.00000, score = 0.00000
sigma = 0.00100, lambda = 1000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000.00000, score = -0.00000
sigma = 0.00100, lambda = 100000.00000, score = 0.00000
sigma = 0.00100, lambda = 1000000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000000.00000, score = 0.00000
sigma = 0.00100, lambda = 100000000.00000, score = 0.00000
sigma = 0.00100, lambda = 1000000000.00000, score = 0.00000
sigma = 0.01000, lambda = 0.00100, score = 0.00000
sigma = 0.01000, lambda = 0.01000, score = 0.00000
sigma = 0.01000, lambda = 0.10000, score = 0.00000
sigma = 0.01000, lambda = 1.00000, score = -0.00000
sigma = 0.01000, lam

/Users/bowenxu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/var/folders/h9/0lx5x32n6vl6m9bsxrssr9l40000gn/T/ipykernel_30656/4228596212.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  Ireg.fit(Z_source, Y_source)


RandomForestRegressor(n_estimators=20)

In [152]:
print(max(sample_density_ratio))
print(min(sample_density_ratio))
mean = np.mean(sample_density_ratio)
print(mean)
sample_density_ratio = sample_density_ratio/mean

8.670116451126331
0.06105860152466247
3.365554690349222


In [137]:
PCRtest(Y_target, X_target, Z_target,V_target, L = 10, K = 30, covariate_shift = False, density_ratio = sample_density_ratio, regr = Ireg)

TypeError: PCRtest() got an unexpected keyword argument 'reg'

In [140]:
PCRtest(Y_source, X_source, Z_source,V_source, L = 5, K = 20, covariate_shift = False, density_ratio = sample_density_ratio, regr = reg)

(array([215., 197., 179., 224., 185.]), 7.38)

In [ ]:
chi_squared_p_value(10, 4)

In [154]:
# verification by the p value
l = 20
count = 0
for j in trange(500):
    
    cov1 = generate_cov_matrix(Y_source, X_source, Z_source,V_source, L = l, K =30, density_ratio = sample_density_ratio, regr = reg)
    w, statistic = PCRtest(Y_source, X_source, Z_source,V_source, L = l, K = 30, covariate_shift = True, density_ratio = sample_density_ratio, regr = reg)
    print(statistic)
    p_value = monte_carlo_p_value(100000, cov1, l, statistic)
    print(p_value)
    if p_value < 0.1:
        count += 1
probability = count/500


  0%|          | 0/500 [00:00<?, ?it/s]

19.20178564502482


  0%|          | 1/500 [00:56<7:46:30, 56.09s/it]

0.74329
21.349408390758235


  0%|          | 2/500 [01:53<7:52:04, 56.88s/it]

...

0.98361
21.269945790550565


100%|█████████▉| 499/500 [7:52:15<00:56, 56.92s/it]

0.64203
23.973562094531143


100%|██████████| 500/500 [7:53:12<00:00, 56.78s/it]

0.5019100000000001


In [ ]:
PCRtest(Y_source, X_source, Z_source, V_source, L = 10, K = 20, covariate_shift = True)

In [ ]:
*write the function

In [ ]:
def multiple_test():

In [ ]:
# previous code

# def int_(x):
#     if x >= 80: return 79
#     return int(x)

# def Many_Tests(m, Alpha):
#     X = []
#     n, p=500, 20
#     for i in trange(m):
#         s = np.random.normal(0, 1, p)
#         t = s + np.random.normal(0, 0.1, p)
#         u = np.random.normal(0, 1, p)
#         Y_source, X_source, V_source, Z_source, Y_target, X_target, V_target, Z_target = generate(n, p, s, t, u, Alpha)
#         u, v = PCRtest(Y_source, X_source, Z_source, L = 5, K = 20, covariate_shift = False)
#         X.append(v)
#     return X
# def Density_Variance(n):
#     a, b=0, 0
#     for i in range(n):
#         z = np.random.normal(0, 1, p)
#         x = z @ s + np.random.normal(0, 1, 1)
#         a += Covariate_Shift_Weight(x, z)**2
#         b += np.exp((z @ t - z @ s)**2)
#     return a/n, b/n

# Power simulation

1.First, we set the rejection threshold to be 0.05. We will run 1000 simulations to estimate the power and average the results over 1000 trials. Plot the power w.r.t L with fixed K = 50.

In [ ]:
# simulation code
K = 20
L = 10
n, p = 1000, 20
s = np.random.normal(0, 1, p)
t = s + np.random.normal(0, 0.1, p)
u = np.random.normal(0, 1, p)

result = []
for l in range(4, 30):
    for i in range
    cov1 = generate_cov_matrix(Y_source, X_source, Z_source, L = l, K = 20)
    result.append(monte_carlo_p_value(100000, cov1, l, PCRtest(Y_source, X_source, Z_source, L = l, K = 20, covariate_shift = True)[1]))
    
    